In [1]:
import evotorch
from evotorch.logging import StdOutLogger
from evotorch.algorithms import GeneticAlgorithm
from evotorch.operators import TwoPointCrossOver, PolynomialMutation

import IPython.display as ipd
import torch

from ohc.vst import VSTHost
from ohc.fitness.fitness import FitnessFunction
from ohc.fitness.clap import CLAPSimilarity

%load_ext autoreload
%autoreload 2

/Users/svanka/miniforge3/envs/ohc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-25 14:23:28,133	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [9]:
PLUGIN_PATH = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"
SAMPLE_RATE = 48000
BLOCK_SIZE = 512

In [10]:
vst = VSTHost(PLUGIN_PATH, "fixed", SAMPLE_RATE, BLOCK_SIZE)

error: attempt to map invalid URI `/Library/Audio/Plug-Ins/VST3/Dexed.vst3'
2024-02-25 14:56:40,365	INFO worker.py:1724 -- Started a local Ray instance.


In [11]:
params = vst.list_params(filter_midi_cc=True)
active_params = [p for p in params if p != "Bypass"]
vst.set_active_params(active_params)

In [12]:
print("Active parameters:", active_params)

Active parameters: ['Cutoff', 'Resonance', 'Output', 'MASTER TUNE ADJ', 'ALGORITHM', 'FEEDBACK', 'OSC KEY SYNC', 'LFO SPEED', 'LFO DELAY', 'LFO PM DEPTH', 'LFO AM DEPTH', 'LFO KEY SYNC', 'LFO WAVE', 'TRANSPOSE', 'P MODE SENS.', 'PITCH EG RATE 1', 'PITCH EG RATE 2', 'PITCH EG RATE 3', 'PITCH EG RATE 4', 'PITCH EG LEVEL 1', 'PITCH EG LEVEL 2', 'PITCH EG LEVEL 3', 'PITCH EG LEVEL 4', 'OP1 EG RATE 1', 'OP1 EG RATE 2', 'OP1 EG RATE 3', 'OP1 EG RATE 4', 'OP1 EG LEVEL 1', 'OP1 EG LEVEL 2', 'OP1 EG LEVEL 3', 'OP1 EG LEVEL 4', 'OP1 OUTPUT LEVEL', 'OP1 MODE', 'OP1 F COARSE', 'OP1 F FINE', 'OP1 OSC DETUNE', 'OP1 BREAK POINT', 'OP1 L SCALE DEPTH', 'OP1 R SCALE DEPTH', 'OP1 L KEY SCALE', 'OP1 R KEY SCALE', 'OP1 RATE SCALING', 'OP1 A MOD SENS.', 'OP1 KEY VELOCITY', 'OP1 SWITCH', 'OP2 EG RATE 1', 'OP2 EG RATE 2', 'OP2 EG RATE 3', 'OP2 EG RATE 4', 'OP2 EG LEVEL 1', 'OP2 EG LEVEL 2', 'OP2 EG LEVEL 3', 'OP2 EG LEVEL 4', 'OP2 OUTPUT LEVEL', 'OP2 MODE', 'OP2 F COARSE', 'OP2 F FINE', 'OP2 OSC DETUNE', 'OP2

In [13]:
clap = CLAPSimilarity(sample_rate=SAMPLE_RATE)

Initializing CLAP model: laion/larger_clap_general


config.json: 100%|██████████| 643/643 [00:00<00:00, 1.59MB/s]
pytorch_model.bin: 100%|██████████| 776M/776M [00:50<00:00, 15.2MB/s] 


Initializing CLAP tokenizer: laion/larger_clap_general


tokenizer_config.json: 100%|██████████| 1.36k/1.36k [00:00<00:00, 12.5MB/s]
vocab.json: 100%|██████████| 798k/798k [00:00<00:00, 2.61MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 2.03MB/s]
tokenizer.json: 100%|██████████| 2.11M/2.11M [00:00<00:00, 8.95MB/s]
special_tokens_map.json: 100%|██████████| 280/280 [00:00<00:00, 2.72MB/s]


Initializing CLAP feature extractor: laion/larger_clap_general


preprocessor_config.json: 100%|██████████| 541/541 [00:00<00:00, 3.28MB/s]


In [14]:
text_target = ["dog bark"]
fitness = FitnessFunction(vst, clap, text_target=text_target)

In [15]:
problem = evotorch.Problem(
    ["max"] * len(text_target),
    fitness.compute,
    solution_length=len(active_params),
    initial_bounds=(0, 1),
    bounds=(0, 1),
)

[2024-02-25 14:57:56] INFO     <80559> evotorch.core: Instance of `Problem` (id:6096846960) -- The `dtype` for the problem's decision variables is set as torch.float32
[2024-02-25 14:57:56] INFO     <80559> evotorch.core: Instance of `Problem` (id:6096846960) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2024-02-25 14:57:56] INFO     <80559> evotorch.core: Instance of `Problem` (id:6096846960) -- The `device` of the problem is set as cpu
[2024-02-25 14:57:56] INFO     <80559> evotorch.core: Instance of `Problem` (id:6096846960) -- The number of actors that will be allocated for parallelized evaluation is 0


In [ ]:
# x = problem.generate_batch(16)

In [ ]:
# fitness.compute(x.values)

In [16]:
ga = GeneticAlgorithm(
    problem,
    popsize=25,
    operators=[
        TwoPointCrossOver(problem, tournament_size=4),
        PolynomialMutation(problem),
    ],
    re_evaluate=False,
)

In [17]:
_ = StdOutLogger(ga)  # Report the evolution's progress to standard output
ga.run(25)  # Run the algorithm for 100 generations
print("Solution with best fitness ever:", ga.status["best"])

AttributeError: 'builtin_function_or_method' object has no attribute 'ndim'

In [ ]:
ga.status["best"][0].shape

In [ ]:
best_discovered_solution = ga.status['best']

In [ ]:
audio, _ = vst.render_now(best_discovered_solution.values.unsqueeze(0), 48, 1.0, 1.0)[0]

In [ ]:
ipd.Audio(audio, rate=SAMPLE_RATE)